In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import  MultiLabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, Model ,Sequential, metrics

2025-11-13 23:45:08.173520: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-13 23:45:14.699117: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-13 23:45:22.752541: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
movies = pd.read_csv('/home/vihan-tandon/Desktop/Movie_Recommender/movie_dataset/movies.csv')
ratings = pd.read_csv('/home/vihan-tandon/Desktop/Movie_Recommender/movie_dataset/ratings.csv',usecols = ['userId','movieId','rating']) 
print(f"Movie shape{movies.shape}")
print(f"Ratings shape{ratings.shape}")
# print(movies.head())
# print(ratings.head())

Movie shape(9742, 3)
Ratings shape(100836, 3)


In [4]:
movies['genres'] = movies['genres'].str.split('|')
mlb = MultiLabelBinarizer()

genre_onehot = mlb.fit_transform(movies['genres'])
genre_df = pd.DataFrame(genre_onehot ,columns=mlb.classes_)
movies = pd.concat([movies,genre_df],axis=1)
# print(movies.head())

In [5]:
merged_data = pd.merge(ratings,movies, on = 'movieId')
# print(merged_data.head())

In [6]:
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

merged_data['user_idx'] = user_encoder.fit_transform(merged_data['userId'])
merged_data['movie_idx'] = movie_encoder.fit_transform(merged_data['movieId'])
# print(merged_data.head())

In [7]:
train_data,test_data = train_test_split(merged_data,test_size=0.2,random_state=42)

genre_columns = mlb.classes_

user_train = train_data['user_idx'].values
movie_train = train_data['movie_idx'].values
genre_train = train_data[genre_columns].values
y_train = train_data['rating'].values

user_test = test_data['user_idx'].values
movie_test = test_data['movie_idx'].values
genre_test = test_data[genre_columns].values
y_test = test_data['rating'].values

print(f"Train data shape: {user_train.shape}, {movie_train.shape}, {genre_train.shape}, {y_train.shape}")
print(f"Test data shape: {user_test.shape}, {movie_test.shape}, {genre_test.shape}, {y_test.shape}")

Train data shape: (80668,), (80668,), (80668, 20), (80668,)
Test data shape: (20168,), (20168,), (20168, 20), (20168,)


# Neural Network(Tower Model)

In [18]:
num_users = merged_data['user_idx'].nunique()
num_movies = merged_data['movie_idx'].nunique()
num_genres = len(genre_columns)

# user_NN = Sequential([
#     layers.Dense(128, activation='relu'),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(32)
# ],name = 'user_nn')

# item_NN = Sequential([
#     layers.Input(shape=(1,)),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(32)
# ],name = 'item_nn')

user_input = layers.Input(shape=(1,), name='user_input')        
user_embedding = layers.Embedding(num_users, 32)(user_input)    
vu = layers.Flatten()(user_embedding)

movie_input = layers.Input(shape=(1,), name='movie_input')      
movie_embedding = layers.Embedding(num_movies, 32)(movie_input) 
m = layers.Flatten()(movie_embedding)


genre_input = layers.Input(shape=(num_genres,), name='genre_input')  
movie_concat = layers.Concatenate()([m, genre_input])        
vm = layers.Dense(32, activation='relu')(movie_concat)

output = layers.Dot(axes=1)([vu,vm])

model = Model(inputs=[ user_input,movie_input,genre_input],outputs=output)
model.compile(optimizer='adam',loss='mse',metrics=['mae'])

model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ movie_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_13        │ (None, 1, 32)     │    311,168 │ movie_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_13          │ (None, 32)        │          0 │ embedding_13[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ genre_input         │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_12        │ (None, 1, 32)     │     19,520 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 52)        │          0 │ flatten_13[0][0], │
│ (Concatenate)       │                   │            │ genre_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_12          │ (None, 32)        │          0 │ embedding_12[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 32)        │      1,696 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_6 (Dot)         │ (None, 1)         │          0 │ flatten_12[0][0], │
│                     │                   │            │ dense_24[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 332,384 (1.27 MB)

 Trainable params: 332,384 (1.27 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
user_train_input = user_train  # shape: (num_samples,)
user_test_input  = user_test
movie_train_input = movie_train  # shape: (num_samples,)
movie_test_input  = movie_test

In [20]:
history = model.fit(
    [user_train_input, movie_train_input, genre_train],
    y_train,
    batch_size=128,
    epochs=10,
    validation_data=([user_test_input, movie_test_input, genre_test], y_test),
    verbose=1
)

Epoch 1/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3.5011 - mae: 1.3967 - val_loss: 0.9131 - val_mae: 0.7383
Epoch 2/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7836 - mae: 0.6831 - val_loss: 0.8281 - val_mae: 0.7010
Epoch 3/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7079 - mae: 0.6469 - val_loss: 0.8093 - val_mae: 0.6916
Epoch 4/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6737 - mae: 0.6292 - val_loss: 0.7935 - val_mae: 0.6815
Epoch 5/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6485 - mae: 0.6158 - val_loss: 0.7769 - val_mae: 0.6782
Epoch 6/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6276 - mae: 0.6060 - val_loss: 0.7696 - val_mae: 0.6750
Epoch 7/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6082 - mae: 0.5953 - val_loss: 0.7659 - val_mae: 0.6717
Epoch 8/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5880 - mae: 0.5847 - val_loss: 0.7497 - val_mae: 0.6675
Epoch 9/10
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - lo

In [26]:
# Suppose you have
movie_idx = 20
user_idx = 10
# Convert movie_idx (internal index) back to movieId
movie_id = movie_encoder.inverse_transform([movie_idx])[0]

# Now, locate the right genre vector for this movieId
genre_vector = movies.loc[movies['movieId'] == movie_id, genre_columns].values

# Reshape and predict
user_array = np.array([[user_idx]])
movie_array = np.array([[movie_idx]])
genre_array = genre_vector  # shape: (1, num_genres)

rating_pred = model.predict([user_array, movie_array, genre_array])
print(f"Predicted rating: {rating_pred[0][0]:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted rating: 3.67
